# Session

Hello everyone! Here we will explore session in langchain. Session is a concept that allows you to maintain state across multiple interactions with a language model. Things we'll cover are:

1. Basic Session using conversation messages
2. Using `InMemoryChatHistory`
3. Use external database

In [91]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model="gpt-3.5-turbo")

## 1 - Session by maintaining conversation messages

In [92]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

sessions = {}  # Use a list to store messages

system_message = SystemMessage(content="You are a helpful AI assistant.")

def get_session(session_id: int):
    if  session_id not in sessions:  
        sessions[session_id] = [system_message]
    return sessions[session_id]

def ask_for_session(session_id, question):
    session = get_session(session_id)
    session.append(HumanMessage(content=question)) 
    response = model.invoke(session)
    session.append(response)
    return response

In [93]:
print("\nSession 0 -----------------")
response = ask_for_session(0, "My name is John") 
print(response.content)
response = ask_for_session(0, "Who is my name?")
print(response.content)

print("\nSession 1 -----------------")
response = ask_for_session(1, "Who is my name?") # different session
print(response.content)

print("\nSummary -------------------")
sessions


Session 0 -----------------
Hello, John! How can I assist you today?
Your name is John.

Session 1 -----------------
I'm sorry, I don't have access to personal information about you. How can I assist you today?

Summary -------------------


{0: [SystemMessage(content='You are a helpful AI assistant.'),
  HumanMessage(content='My name is John'),
  AIMessage(content='Hello, John! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ada7949-65b0-47fd-8ece-d7abe645a0f6-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33}),
  HumanMessage(content='Who is my name?'),
  AIMessage(content='Your name is John.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 46, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c8f7acc5-4f1a-4457-9651-b93d35786b06-0', usage_metadata={'input_tokens': 46, 'output_tokens': 5, 'total_tokens': 51})],
 1: [SystemMessage(content='You are a helpful AI assistant.'),
  Human

## 2 - Session using `InMemoryChatMessageHistory`


We can create session using langchain class `InMemoryChatMessageHistory`. This will automatically append the human and assistant messages to the currect conversation.

To begin, let's create a chain, consists of prompt and model

In [94]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt =  ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant. Reply messages with Indonesian language"),
    ('user', '{messages}')
])

model = ChatOpenAI(model="gpt-3.5-turbo")

chain = prompt | model| StrOutputParser()

Next, we create `get_session` function that retrivet session from store. Note that now the session is `InMemoryChatMessageHistory` object.

In [95]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)

# Session
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

and lastly, let's wrap our `chain` and `get_session_history` in class `RunnableWithMessageHistory`

In [96]:
from langchain_core.runnables import RunnableWithMessageHistory

app = RunnableWithMessageHistory(chain, get_session_history)

Now, we can invoke the app with by feeding messages and config that specify session. 

In [97]:
config = {"configurable": {"session_id": "1"}}

In [98]:
response = app.invoke(
    {"messages": HumanMessage(content="Hi! I'm Bob. Who are you?")},
    config=config,
)

store['1'].messages

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


[HumanMessage(content="Hi! I'm Bob. Who are you?"),
 AIMessage(content='Halo Bob! Saya adalah asisten virtual yang siap membantu Anda. Ada yang bisa saya bantu?')]

Behind the scene, app invocation will cause:

1. `RunnableWithMessageHistory` searches for a session using the `get_session` function.
2. If the session is empty, it appends the session with the full prompt. If there is already a session, it will be appended only with the message (human) somehow.

In [99]:
response = app.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response
store['1'].messages

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


[HumanMessage(content="Hi! I'm Bob. Who are you?"),
 AIMessage(content='Halo Bob! Saya adalah asisten virtual yang siap membantu Anda. Ada yang bisa saya bantu?'),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Halo Bob! Saya adalah asisten virtual yang siap membantu Anda. Nama Anda adalah Bob, ada yang bisa saya bantu?')]

In [100]:
config = {"configurable": {"session_id": "2"}}

response = app.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response
store['2'].messages


Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


[HumanMessage(content="What's my name?"),
 AIMessage(content='Maaf, saya tidak punya akses ke informasi pribadi seperti nama Anda. Ada yang bisa saya bantu?')]

## Session `InMemoryChatMessageHistory` with Two Arguments

OK. Now what if we want to pass two arguments to the prompt? It is okey, as long as you define which one is messages_key because it will be appended to the session!

In [106]:
# create prompt that has two arguments
prompt =  ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant. Reply messages in {language}"),
    ('user', '{messages}')
])

chain = prompt | model| StrOutputParser()

app = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages") # specify the messages key


In [107]:
config = {"configurable": {"session_id": "4"}}

app.invoke({
    "language": "Indonesian",
    "messages": "Hello"
}, config=config)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Halo! Ada yang bisa saya bantu?'

The syntatic sugar (`{some_key}`) in prompt only works in string. If you want to pass list of messages as an argument, use `MessagesPlaceholder`

In [104]:
from langchain_core.prompts import MessagesPlaceholder

# create a prompt that has messages as an argument
prompt =  ChatPromptTemplate.from_messages([
    ('system', "You are a helpful assistant. Reply messages in {language}"),
    MessagesPlaceholder("message_list") # message list will be spred here
])

chain = prompt | model| StrOutputParser()

app = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="message_list") # specigy the messages_key

In [105]:
config = {"configurable": {"session_id": "10"}}

app.invoke({
    "language": "Indonesian",
    "message_list": [("human", "My name is Sukses"), ("system", "Give them a praise on his name")]
}, config=config)

store["10"].messages

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


[('human', 'My name is Sukses'),
 ('system', 'Give them a praise on his name'),
 AIMessage(content='Halo Sukses! Nama Anda memiliki makna yang sangat positif dan inspiratif. Semoga Anda selalu meraih kesuksesan dalam segala hal yang Anda lakukan. Apakah ada yang bisa saya bantu hari ini?')]